In [55]:
import os
from os.path import abspath
import findspark
findspark.init()

In [56]:
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-19"
os.environ["SPARK_HOME"] = r"C:\spark\spark-3.3.2"
os.environ["HADOOP_CONF_DIR"] = r"C:\spark\hadoop-3.3.5\etc\hadoop"
os.environ["YARN_CONF_DIR"] = r"C:\spark\hadoop-3.3.5\etc\hadoop"
os.environ["YARN_HOME"] = r"C:\spark\hadoop-3.3.5\etc\hadoop"
os.environ["HADOOP_HOME"] = r"C:\spark\spark-3.3.2\hadoop\bin"



In [57]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
import tempfile


warehouse_location = abspath('spark-warehouse')



In [58]:
conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.submit.deployMode","client")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.yarn.am.memory","1g")
conf.set("spark.eventLog.enabled ","true")
conf.set("spark.sql.adaptive.enabled","true")
conf.set('spark.driver.extraClassPath', r"C:\spark\spark-3.3.2\mssql-jdbc-9.4.0.jre11.jar")
conf.set('spark.executor.extraClassPath', r"C:\spark\spark-3.3.2\mssql-jdbc-9.4.0.jre11.jar")
spark = SparkSession.builder\
    .config(conf=conf)\
    .config("spark.sql.warehouse.dir", warehouse_location)\
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

In [59]:
#/usr/hdp/current/spark-client/bin/spark-submit  --class org.apache.spark.examples.SparkPi  --master yarn   --deploy-mode cluster   --executor-memory 1G   --num-executors 3   /usr/hdp/current/spark-client/lib/spark-examples*.jar   100


In [60]:
#--class org.apache.spark.examples.SparkPi --master yarn --deploy-mode client --driver-memory 4g --num-executors 4 --executor-memory 4g --executor-cores 4 /opt/apps/spark-1.6.0-bin-hadoop2.6/lib/spark-examples*.jar 10


In [61]:
spark

In [62]:
sc = sparkContext=spark.sparkContext
path = "C:\pyspark_cloudera_testes\testecomcsv_neo.xlsx"


In [63]:
distFile = sc.textFile(path)

In [64]:
distFile

C:\pyspark_cloudera_testes	estecomcsv_neo.xlsx MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:


import socket
import tempfile
import threading
import zipfile
from configparser import NoOptionError
from livy.client import HttpClient

session_id = 0
job_id = 1
# Make sure host name is lower case. See LIVY-582
base_uri = 'http://{0}:{1}'.format(socket.gethostname().lower(), 8998)
client_test = None
invoked_queued_callback = False
invoked_running_callback = False

In [65]:
import json, pprint, requests, textwrap
host = 'http://localhost:8998'
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 1,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'spark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [66]:
import requests

url = "http://localhost:8998/sessions"

payload={}
headers = {
  'X-Requested-By': 'admin'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

{"from":0,"total":2,"sessions":[{"id":0,"name":null,"appId":null,"owner":null,"proxyUser":null,"state":"dead","kind":"spark","appInfo":{"driverLogUrl":null,"sparkUiUrl":null},"log":["\tat org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1111)","\tat org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1120)","\tat org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scala)","Caused by: java.lang.ClassNotFoundException: scala.Function0$class","\tat java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)","\tat java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)","\tat java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)","\t... 20 more","23/04/16 02:11:48 INFO ShutdownHookManager: Shutdown hook called","23/04/16 02:11:48 INFO ShutdownHookManager: Deleting directory /tmp/spark-128bcf5d-9b66-43b9-aa64-b99c024c05e0"]},{"id":1,"name":null,"appId":null,"owner":null,"proxyUser":null,"state":"starting","kind":"spark","appInfo":{"

In [88]:
url = "http://localhost:8998/sessions"


headers = {
  'X-Requested-By': 'admin',
  'Content-Type': 'application/json'
}


data = {
  'code': textwrap.dedent("""
    val NUM_SAMPLES = 100000;
    val count = sc.parallelize(1 to NUM_SAMPLES).map { i =>
      val x = Math.random();
      val y = Math.random();
      if (x*x + y*y < 1) 1 else 0
    }.reduce(_ + _);
    println(\"Pi is roughly \" + 4.0 * count / NUM_SAMPLES)
    """)
}

r = requests.post(url, data=json.dumps(data), headers=headers)
r.json()




'Unrecognized field "code" (class org.apache.livy.server.interactive.CreateInteractiveRequest), not marked as ignorable (15 known properties: "executorCores", "conf", "driverMemory", "name", "driverCores", "pyFiles", "archives", "kind", "files", "jars", "proxyUser", "numExecutors", "heartbeatTimeoutInSecond", "queue", "executorMemory"])\n at [Source: (org.eclipse.jetty.server.HttpInputOverHTTP); line: 1, column: 11] (through reference chain: org.apache.livy.server.interactive.CreateInteractiveRequest["code"])'

In [67]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{u'state': u'idle', u'id': 0, u'kind': u'spark'}

{'state': 'idle', 'id': 0, 'kind': 'spark'}

In [68]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

{u'output': None, u'state': u'running', u'id': 0}

{'output': None, 'state': 'running', 'id': 0}

In [ ]:
statement_url = host + r.headers['location']
r = requests.get(statement_url, headers=headers)
pprint.pprint(r.json())

{u'id': 0,
  u'output': {u'data': {u'text/plain': u'res0: Int = 2'},
              u'execution_count': 0,
              u'status': u'ok'},
  u'state': u'available'}

In [ ]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

{u'output': None, u'state': u'running', u'id': 0}

{'output': None, 'state': 'running', 'id': 0}

In [70]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

{u'output': None, u'state': u'running', u'id': 0}

{'output': None, 'state': 'running', 'id': 0}

In [ ]:
#Post
url = "http://localhost:8999/batches"

payload = json.dumps({
  "file": "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/o2ov2.coding.keywords-1.0.0.jar",
  "className": "com.nielseniq.o2ov2.keywords.KeywordsCodingApp",
  "jars": [
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2CommonLibs/*",
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/o2ov2.common-1.0.0.jar",
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/CommonInfra-2.10.0-O2OV2.jar",
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/DataValidator-2.10.0-O2OV2.jar",
    "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/KeywordsCoding-2.10.0-O2OV2.jar"
  ],
  "name": "DEV_O2OV2_KeywordsCoding_20221410",
  "driverMemory": "4g",
  "executorMemory": "4g",
  "numExecutors": 7,
  "executorCores": 4
})
headers = {
  'X-Requested-By': 'admin',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
#Get
url = "http://localhost:8998/sessions"

payload={}
headers = {
  'X-Requested-By': 'admin'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)
